# Llama3 한국어 데이터 파인튜닝


In [ ]:
!pip install datasets
!pip install accelerate
!pip install peft

In [ ]:
import huggingface_hub

huggingface_hub.login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "meta-llama/Meta-Llama-3-8B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

In [ ]:
model

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_availabe() else "cpu")
device

In [ ]:
# 모델을 GPU에 올리기
model.to(device)

In [ ]:
# 모델 상태 확인 - 답변을 못함
prompt = "안국동에 대해 알려줘"
inputs = tokenizer(prompt, return_tensors='pt').to(device)

generate_ids=model.generate(inputs.input_ids, max_length=50, attention_mask=inputs.attenstion_mask)
tokenizer.batch_decoe(generate_ids, skip_special_tokens=True, clean_up_tokenization_space=False)[0]

In [ ]:
from datasets import load_dataset, DatsetDic
raw_dataset = load_dataset("nlpai-lab/kullm-v2", split="train") # 허깅페이스 한국어 관련 데이터셋

In [ ]:
raw_dataset

In [ ]:
raw_dataset["output"][:10]

In [ ]:
# 샘플 dataset 출력 5만줄 가져오기
sampled_dataset = raw_dataset.select(range(50000))

In [ ]:
# 토크나이저 meta는 범용적이므로 한국어 특화 모델을 이용하는 것이 좋음
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('psymon/KoLlama2-7b')
#tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

In [ ]:
# Tokenizing 확인
sampled_text = "반갑습니다"
tokenizer.tokenize(sampled_text)

# 숫자로 변환, 문장 시작 토큰은 1, 문장 끝 토큰은 2인 경우가 많음
tokenizer(sampled_text, return_length=True)


In [ ]:
context_length = 128

def tokenize(batch) :
    outputs = tokenizer(
        batch['output'],
        max_length=context_length,
        truncation=True,
        return_overflowing_tokens=True,
        return_length=True
    )

    input_batch=[]
    for length, input_ids in zip(outputs['length'], outputs['input_ids']):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}

In [ ]:
# sampled_dataset

tokenized_datasets = sampled_dataset.map(tokenize, batched=True, remove_columns=raw_dataset.column_names)

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

model.to(device)

In [ ]:
# 모델의 상태 확인
prompt = "안국동에 대해 알려줘"
inputs = tokenizer(prompt, return_tensors='pt').to(device)

generate_ids=model.generate(inputs.input_ids, max_length=50, attention_mask=inputs.attenstion_mask)
tokenizer.batch_decoe(generate_ids, skip_special_tokens=True, clean_up_tokenization_space=False)[0]

In [ ]:
# Data Collator 불러오기
from transformers import DataCollatorForLanguageModeling
tokenizer.pd_token=tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [ ]:
# Data Collator에 Data load 확인
out = data_collator([tokenized_datasets[i] for i in range(3)])
out['input_ids'][0][:20], out['attention_mask'][0][:20], out['labels'][0][:20]

In [ ]:
# train arguments 입력
from transformers import TrainingArguments

#batch_size=32
batch_size = 16
#logging_steps = 100
logging_steps = 10
#learing_rate=5e-4
learing_rate=3e-3
num_epochs=1
args = TrainingArguments(
    output_idr='/content/drive/myDrive/gdrive/llama_result/testllama', # 학습 결과(모델. 체크포인트 로그 등)가 저장될 경로 지정
    per_device_train_batch_size=batch_size,       # 학습 시 사용되는 디바이스(예: GPU) 당 배치 크기 지정
    per_device_eval_batch_size=batch_size,        # 평가 시 사용되는 디바이스 당 배치 크기 지정
    logging_steps=logging_steps,                  # 몇 스텝마다 로그를 기록할지 지정
    save_steps=logging_steps,                     # 몇 스텝마다 모델 체크포인트를 저장할지 지정
    gradient_accumulation_steps=8,                # 그라디언트 누적 스텝 수를 지정. 이를 통해 더 큰 가상 배치 크기를 사용
    num_train_epochs=num_epochs,                  # 전체 학습 데이터셋을 몇 번 반복할지 지정
    weight_decay=0.1,                             # 가중치 감쇠율을 지정. 이는 모델의 과적합을 방지하는데 도움
    warmup_steps=logging_steps,                   # 학습 초기의 워밍업 단계에서 사용할 스텝 수 지정
    lr_scheduler_type='cosine',                   # 학습률 스케줄러의 타입을 지정. 여기서는 'cosine' 스케줄러 사용
    learning_rate=learning_rate,                  # 초기 학습률 지정
    fp16=True,                                    # FP16(반 정밀도) 연산을 사용하여 훈련 속도를 높이고 메모리 사용량을 감소
    push_to_hub=False                             # 허깅페이스에 푸시할 지 여부
)


In [ ]:
# 학습기
from transformers import Trainer

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets
)

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
trainer.train()

In [ ]:
# 모델 저장
model.save_pretrianed('pre_llama')
tokenizer.save_pretrained('pre_llama')

In [ ]:
model.push_to_hub('wonik-hi/llama_pre_model')
tokenizer.push_to_hub('wonik/hi/llama_pre_tokenizer')

# 한번 더 트레이닝

In [ ]:
from datasets import load_dataset, DatasetDict

# hugging space에서 dataset 가져오기
raw_dataset = load_dataset("maywell/ko_wikidata_QA", split="train")

In [ ]:
from datasets import load_dataset, DatasetDict

# 데이터 소스 가져오기
raw_dataset = load_dataset("maywell/ko_wikidata_QA", split="train")

# 데이터셋 분할 비율 설정
train_test_split = raw_dataset.train_test_split(test_size=0.1) # 10%를 테스트 세트로 분할

# train 데이터셋에서 다시 10%를 검증 세트(validation set)로 분할
train_validation_split = train_test_split['train'].train_test_split(test_size=0.1)

# 최종 데이터셋 구성
dataset = DatasetDict({
    'train': train_validation_split['train'],
    'test': train_test_split['test'],
    'validation' : train_validation_split['test'],
})

# 각 데이터셋의 크기 확인
dataset_sizes = {split: len(dataset[split]) for split in dataset.keys()}
print(dataset_sizes)

sampled_dataset = DatasetDict(
    {
        "train": dataset['train'].select(range(10000)).shuffle(),
        "valid": dataset['test'].select(range(1000)).shuffle()
    }
)

sampled_dataset


In [ ]:
# 토크나이즈
context_length=128

def tokenize(batch):
    outputs = tokenizer(
        batch['output'],
        max_length=context_length,
        truncation=True,            # true 설정 시, max_lenggth를 초과하는 텍스트 자름
        return_overflowing_tokens=True,
        return_length=True
    )

    input_batch=[]
    for length, input_ids in zip(outputs['length'], outputs['input_ids']):
        if length==context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}

In [ ]:
# 데이터 전체 토크나이징
tokenized_datasets = sampled_dataset.map(tokenize, batched=True, remove_columns=raw_dataset.column_names)

In [ ]:
# PEFT LoRA 불러오기
from transformers import LlamaForCausalLM
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("pre_llama")
model = LlamaForCausalLM.from_pretrianed("pre_llama")

model

In [ ]:
# PEFT 라이브러리
from peft import get_peft_model, LoraConfig, TaskType

In [ ]:
# PEFT Config
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM # 모델의 작업 유형 지정. TaskType.CAUSAL_LM으로 설정하여 인과 언어 모델링 작업 수행
    , inference_model=False      # 추론 모드 설정 (False로 설정하여 학습 모드로 설정)
    , r=4                        # 로우랭크 크기. 매개변수의 효율성을 높이기 위해 사용되는 저차원 행렬의 랭크를 의미.
    , lora_alpha=16              # PEFT의 추론 간섭정도. 로우랭크 행렬의 스케일링 팩터로, 모델의 학습 및 추론 성능에 영향을 미침.
    , lora_dropout=0.1           # 드로방웃 비율 설정. 과적합을 방지하기 위해 뉴런의 일부를 무작위로 비활성화하는 기법
)

model = get_peft_model(model, peft_config)
model.to(device)

In [ ]:
# 파라미터 학습 수 확인
model.print_trainable_parameters()

In [ ]:
# 데이터 콜레이터에 데이터 올리기
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [ ]:
# Training Argument 입력
from transformers import Trainer, TrainingArguments

args=TrainingArguments(
    output_dir="/content/drive/MyDrive/gdrive/llama_result/pre_llama",
    per_device_train_batch_size=4, # 데이터 배치 사이즈
    logging_steps=500,             # 훈련에서 로깅할 단계
    gradient_accumulation_steps=8,  # 8단걔마다 w 조정
    num_train_epochs=1,            # 전체 훈련 데이터세트 반복 횟수
    weight_decay=0.1,              # w를 10%씩 손실을 고의로 일으키며, overfitting을 방지한다.
    lr_scheduler_type='cosine',      # LR 변화를 코사인 함수 형태로 변화
    learning_rate=5e-4,             # 학습률
    save_steps=1000,                # 기록 저장 스텝
    fp16=True,                      # 16비트 부동소수점 연산(True:메모리 사용량 감소, 속도 증가)
    push_to_hub=False,              # 허깅페이스 공유 여부
)

trainer=Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets['train']
)

In [ ]:
# 학습하기
trainer.train()

# 저장하기
model.save_pretrained("/content/drive/MyDrive/gdrive/llama_result/peft_llama_adapter")

In [ ]:
# prellama + Peft

from transformers import LlamaForCausalLM
from peft import PeftModel, PeftConfig
base_model = LlamaForCausalLM.from_pretrained('pre_llama')
model_load=PeftModel.from_pretrained(base_model, 'peft_llama_adapter')
model_load.to(device)


In [ ]:
# 세팅 - 모델들을 머지 하는 방법 : 두 번의 학습효과
model = model_load.merge_and_unload()

In [ ]:
# 모델 용량 확인
import os
os.start('peft_llama_adapter/adapter_model.safetensors').st_size/(1024*1924)

In [ ]:
# 최종 모델에 쿼리 날려보기
question = "알고리즘 분석"
prompt=f"""{question}"""
inputs = tokenizer(prompt, return_tensors='pt')
inputs.to(device)
generate_ids = model.generate(inputs.input_ids, max_length=100)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

In [ ]:
!pip install sentence-transformers